In [1]:
import os

In [2]:
%pwd

'd:\\project\\chicken-fecal-classification_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\project\\chicken-fecal-classification_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBasmodelconfig:
    root_dir =Path
    base_model_apth = Path
    update_base_model_path = Path
    param_image_size = Path
    param_leaening_rate: float
    params_include_top: bool
    params_weights : str
    param_classes : int

In [6]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create the directories specified in the config file
        create_directories([Path(self.config["artifacts_root"])])

    # Define any additional methods here, for example:
    def get_data_ingestion_config(self):
        # Example method to retrieve configuration details
        data_config = self.config.get("data_ingestion", {})
        return data_config


In [8]:
from pathlib import Path

class configurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ) -> None:
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(params_filepath)

        # Debug print to verify structure of self.config
        print("Loaded config:", self.config)  # Check if 'data_ingestion' and 'artifacts_root' keys exist

        # Convert `artifacts_root` to Path object before passing to `create_directories`
        create_directories([Path(self.config["artifacts_root"])])

    def get_data_ingestion_config(self) -> PrepareBasmodelconfig:

        config = self.config.prepare_base_model

        prepare_base_model_config = PrepareBasmodelconfig(
            root_dir = path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            params_image_size = self.param.IMAGE_SIZE,
            paramslearning_rate = self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            param_classes=  self.params.CLASSES
        )

        return prepare_base_model_config
        
        
    
        

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [10]:
import tensorflow as tf
from pathlib import Path

class PrepareBasemodel:
    def __init__(self, config):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.param_include_top
        )
        self.save_model(path=self.config.base_model_path, model=self.model)

    def save_model(self, path: Path, model: tf.keras.Model):
        model.save(path)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif freeze_till is not None and freeze_till > 0:
            for layer in model.layers[:freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)


In [16]:
from cnn_classifier.entity.config_entity import DataIngestionConfig,PrepareBaseModelConfig
from cnn_classifier.entity.config_manager import ConfigurationManager

try:
    config = ConfigurationManager()  # Initialize configuration manager
    prepare_base_model_config = config.get_prepare_base_model_config()  # Get base model config

    # Now you can use the configuration to pass into your model preparation
    prepare_base_model = PrepareBasemodel(config=prepare_base_model_config)

    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()

except Exception as e:
    print(f"An error occurred: {e}")
    raise e


[2024-11-14 17:51:05,859: common: YAML file: d:\project\chicken-fecal-classification_project\config\config.yaml loaded successfully]
[2024-11-14 17:51:05,861: common: Content of d:\project\chicken-fecal-classification_project\config\config.yaml: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_injection', 'source_url': 'https://github.com/itisha249/chicken-fecal-classification_project/raw/refs/heads/main/archive%20(1).zip', 'local_data_file': 'artifacts/data_injection/data.zip', 'unzip_dir': 'artifacts/data_injection', 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}}}]
[2024-11-14 17:51:05,865: common: YAML file: d:\project\chicken-fecal-classification_project\param.yaml loaded successfully]
[2024-11-14 17:51:05,867: common: Content of d:\project\chicken-fecal-classification_project\param.yaml:

BoxKeyError: "'prepare_base_model'"